In [5]:
from Utils import *
from classifier.CNNClassifier import CNNClassifier
from classifier.Classifier import Classifier
from classifier.ConstantClassifier import ConstantClassifier
from classifier.LinearClassifier import LinearClassifier
from classifier.RFClassifier import RFClassifier
from classifier.SNNClassifier import SNNClassifier
from Settings import *
from sklearn.model_selection import train_test_split
from main import run_for_classifier

SAVE=True
LOAD=False

### Preliminary informations
##### Disclaimer : The neural networks results may vary a bit when re-training them due to some randomness in tensorflow-gpu / cuDNN.

The models were trained on a computer with an i7-6700K, 16GB RAM and a GTX 1070.    
The main libraries used are Keras with tensorflow backend, librosa, scipy.    
###### If you want to avoid retraining every classifier, set LOAD=True above. The RF with 100 estimators couldn't be pushed to the repository due to its file size though.

Let's begin by explaining how to measure the performance of the classifiers. There are two main metrics :    
- The sample accuracy : It's the accuracy for predicting the label of a sample of a file. For the CNN, it's a window of 10 samples.
- The file accuracy : It's the accuracy for predicting the label of a file. The label is simply calculated by taking the most predicted label on all the samples of the file.

The files were cut in samples of 20 MFCC features using the librosa library. This was mostly sufficient to allow the training on most of the classifiers. No preprocessing was done on the audio files or the MFCC features except normalizing them.    
Some more preprocessing was done to allow the files to be fed to a CNN (cutting the files in windows).

The files were separated in train set and test set using 80/20 proportions. There are 2703 files in total.

In [2]:
features_with_label = files_to_features_with_labels(list_files(AUDIO_FILES_DIR))
train_set, test_set = train_test_split(features_with_label, random_state=SEED, train_size=TRAIN_PERCENT, test_size=1-TRAIN_PERCENT)

def test_classifier(classifier : Classifier) -> None:
    """
    Wrapper for method run_for_classifier.
    :param classifier: The classifier to test
    """
    one_d = not isinstance(classifier, CNNClassifier)
    run_for_classifier(classifier, test_set=test_set, train_set=train_set, save=SAVE, load=LOAD, one_d=one_d)

### Constant Classifier
This classifier always predicts male.

In [3]:
test_classifier(ConstantClassifier())

Finished loading/creating features
Using classifier ConstantClassifier
Training ConstantClassifier
Saved ConstantClassifier
Predicting on files...
Predicting on samples...
Test accuracy - files : 0.4824399260628466
Test accuracy - samples : 0.5065437497537723


This gives us some information on the files as well as a baseline :     
- The files are distributed nearly uniformly between male and female
- When taking duration into account, the files are distributed even more closely half male / half female

This means that it is not really needed to perform balancing between the train and test set, assuming we distribute the files randomly between them.

### Linear SVC
This classifier is created using SVM with a linear kernel. It could serve as another simple baseline.

In [4]:
test_classifier(LinearClassifier(c=1))

Finished loading/creating features
Using classifier LinearClassifier - C 1
Training LinearClassifier - C 1
Saved LinearClassifier - C 1
Predicting on files...
Predicting on samples...
Test accuracy - files : 0.822550831792976
Test accuracy - samples : 0.7052279084426585


 Here, the linear SVC returns quite mediocre results, having a file-accuracy of only 82%, and a worse sample accuracy of 70%.    
Trying to increase the C parameter to 1000 makes the classifier not converge (or very slowly. When tested earlier, it hadn't converged in 10000 iterations).    
It isn't surprising that a linear classifier would fail on this problem, as it is probably non-linear. Improving the preprocessing as well as doing features selection could improve the result.

In [5]:
test_classifier(LinearClassifier(c=1000))

Finished loading/creating features
Using classifier LinearClassifier - C 1000
Training LinearClassifier - C 1000
Saved LinearClassifier - C 1000
Predicting on files...
Predicting on samples...
Test accuracy - files : 0.5508317929759704
Test accuracy - samples : 0.5846669030453453


### RandomForestClassifier
Let's test RandomForestClassifiers, which are known to be quite accurate when features are well-defined and results are dependent on these features, which should be the case with the MFCC features and the speaker's gender.

In [6]:
test_classifier(RFClassifier(n_estimators=10))

Finished loading/creating features
Using classifier RFClassifier - n_est 10 - max_depth None
Training RFClassifier - n_est 10 - max_depth None
Saved RFClassifier - n_est 10 - max_depth None
Predicting on files...
Predicting on samples...
Test accuracy - files : 0.9981515711645101
Test accuracy - samples : 0.8465665996927078


Here, we can already see that the files accuracy is nearly perfect (in fact, only 1 file is mispredicted). The sample accuracy could be better though.    
Let's try to improve it by increasing the number of trees. We may also get a 100% file accuracy by doing this.

In [7]:
test_classifier(RFClassifier(n_estimators=100))

Finished loading/creating features
Using classifier RFClassifier - n_est 100 - max_depth None
Training RFClassifier - n_est 100 - max_depth None
Saved RFClassifier - n_est 100 - max_depth None
Predicting on files...
Predicting on samples...
Test accuracy - files : 0.9981515711645101
Test accuracy - samples : 0.8814324547925777


It doesn't seem to improve much and isn't really worth it given the increase in memory and time consumption.    
Let's try with 1 and 5 trees to see if we can go lower than 10 while preserving the accuracy :

In [8]:
test_classifier(RFClassifier(n_estimators=5))
print("\n")
test_classifier(RFClassifier(n_estimators=1))

Finished loading/creating features
Using classifier RFClassifier - n_est 5 - max_depth None
Training RFClassifier - n_est 5 - max_depth None
Saved RFClassifier - n_est 5 - max_depth None
Predicting on files...
Predicting on samples...
Test accuracy - files : 0.9981515711645101
Test accuracy - samples : 0.8261513611472245


Finished loading/creating features
Using classifier RFClassifier - n_est 1 - max_depth None
Training RFClassifier - n_est 1 - max_depth None
Saved RFClassifier - n_est 1 - max_depth None
Predicting on files...
Predicting on samples...
Test accuracy - files : 0.9926062846580407
Test accuracy - samples : 0.746507505023047


Strangely enough, we get a really good result with even only one tree. The sample accuracy decreases quite a bit obviously, but the file accuracy is still very good. This means that there are features which are very relevant to finding the speaker's gender.   
With 5 trees, the file accuracy is the same as the one with 10 trees.    
The RandomForest seems to be very well suited for this problem, as expected. What was less expected was that such a performance was obtained using a very small number of estimators.    
RandomForest is therefore a very efficient and easy way to solve this problem.

### Shallow Neural Network
Let's try using a shallow neural network with only a single fully-connected hidden layer.    
In theory, such a simple neural net should be able to approximate every function perfectly, but in practice, a deep neural network usually yields better results than a wide one.

In [9]:
test_classifier(SNNClassifier(num_units=64, verbose=1))

Finished loading/creating features
Using classifier SNNClassifier - units 64
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                1344      
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 64)                64        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 1,473
Trainable params: 1,473
Non-trainable params: 0
_________________________________________________________________
None
Training SNNClassifier - units 64
Train on 384480 samples, validate on 96121 samples
Epoch 1/300
384480/384480 [==============================] - 9s 24us/step - loss: 0.5344 - acc: 0.7202 - 

Epoch 52/300
384480/384480 [==============================] - 9s 25us/step - loss: 0.3150 - acc: 0.8586 - val_loss: 0.3292 - val_acc: 0.8560
Epoch 53/300
384480/384480 [==============================] - 9s 24us/step - loss: 0.3150 - acc: 0.8592 - val_loss: 0.3339 - val_acc: 0.8537
Epoch 54/300
384480/384480 [==============================] - 10s 25us/step - loss: 0.3144 - acc: 0.8595 - val_loss: 0.3289 - val_acc: 0.8561
Epoch 55/300
384480/384480 [==============================] - 10s 25us/step - loss: 0.3143 - acc: 0.8587 - val_loss: 0.3306 - val_acc: 0.8533
Epoch 56/300
384480/384480 [==============================] - 10s 25us/step - loss: 0.3140 - acc: 0.8594 - val_loss: 0.3264 - val_acc: 0.8574
Epoch 57/300
384480/384480 [==============================] - 9s 23us/step - loss: 0.3131 - acc: 0.8596 - val_loss: 0.3228 - val_acc: 0.8584
Epoch 58/300
384480/384480 [==============================] - 9s 24us/step - loss: 0.3131 - acc: 0.8597 - val_loss: 0.3286 - val_acc: 0.8569
Epoch 59/3

We already get a nearly perfect accuracy for files, which is very impressive with only 64 units. This probably means that the function to approximate / the problem is not really complex, but still non-linear.    
Let's try to improve the accuracy by increasing the number of units in the dense layer.

In [10]:
test_classifier(SNNClassifier(num_units=128, verbose=1))

Finished loading/creating features
Using classifier SNNClassifier - units 128
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               2688      
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 128)               128       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________
None
Training SNNClassifier - units 128
Train on 384480 samples, validate on 96121 samples
Epoch 1/300
384480/384480 [==============================] - 10s 25us/step - loss: 0.5385 - acc: 0.7174

Epoch 52/300
384480/384480 [==============================] - 9s 24us/step - loss: 0.2953 - acc: 0.8684 - val_loss: 0.3134 - val_acc: 0.8621
Epoch 53/300
384480/384480 [==============================] - 9s 24us/step - loss: 0.2943 - acc: 0.8689 - val_loss: 0.3103 - val_acc: 0.8641
Epoch 54/300
384480/384480 [==============================] - 9s 24us/step - loss: 0.2942 - acc: 0.8691 - val_loss: 0.3155 - val_acc: 0.8637
Epoch 55/300
384480/384480 [==============================] - 9s 23us/step - loss: 0.2938 - acc: 0.8694 - val_loss: 0.3257 - val_acc: 0.8564
Epoch 56/300
384480/384480 [==============================] - 9s 24us/step - loss: 0.2934 - acc: 0.8702 - val_loss: 0.3169 - val_acc: 0.8619
Epoch 57/300
384480/384480 [==============================] - 9s 23us/step - loss: 0.2932 - acc: 0.8700 - val_loss: 0.3057 - val_acc: 0.8681
Epoch 58/300
384480/384480 [==============================] - 9s 23us/step - loss: 0.2925 - acc: 0.8704 - val_loss: 0.3155 - val_acc: 0.8610
Epoch 59/300


As expected, the sample accuracy is ~1% better, and the file accuracy is now perfect.

In [11]:
test_classifier(SNNClassifier(num_units=256, verbose=1))

Finished loading/creating features
Using classifier SNNClassifier - units 256
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               5376      
_________________________________________________________________
p_re_lu_3 (PReLU)            (None, 256)               256       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_3 (Activation)    (None, 1)                 0         
Total params: 5,889
Trainable params: 5,889
Non-trainable params: 0
_________________________________________________________________
None
Training SNNClassifier - units 256
Train on 384480 samples, validate on 96121 samples
Epoch 1/300
384480/384480 [==============================] - 10s 25us/step - loss: 0.5143 - acc: 0.7346

384480/384480 [==============================] - 10s 25us/step - loss: 0.2654 - acc: 0.8846 - val_loss: 0.2864 - val_acc: 0.8768
Epoch 52/300
384480/384480 [==============================] - 9s 24us/step - loss: 0.2652 - acc: 0.8846 - val_loss: 0.2880 - val_acc: 0.8768
Epoch 53/300
384480/384480 [==============================] - 9s 24us/step - loss: 0.2651 - acc: 0.8843 - val_loss: 0.2885 - val_acc: 0.8744
Epoch 54/300
384480/384480 [==============================] - 9s 24us/step - loss: 0.2648 - acc: 0.8846 - val_loss: 0.2869 - val_acc: 0.8761
Epoch 55/300
384480/384480 [==============================] - 10s 25us/step - loss: 0.2646 - acc: 0.8849 - val_loss: 0.2859 - val_acc: 0.8773
Epoch 56/300
384480/384480 [==============================] - 9s 24us/step - loss: 0.2645 - acc: 0.8849 - val_loss: 0.2870 - val_acc: 0.8764
Epoch 57/300
384480/384480 [==============================] - 10s 25us/step - loss: 0.2643 - acc: 0.8853 - val_loss: 0.2858 - val_acc: 0.8773
Epoch 58/300
384480/384

We gain a very marginal increase in sample accuracy. File accuracy is still perfect.

I decided to not add any Dropout or regularization to the network because, as we can see in the results, there is little to no overfitting.

### Convolutional Neural Network
The CNN Classifier was more difficult to implement.    
First, the input is different compared to the other classifiers. We need to give identical 2D inputs to the network, but the number of samples per file varies a lot.    
- The first attempt was padding the smaller files to the size of the biggest one by adding 0s features. Due to the difference in size, the network couldn't learn anything, as most of the data was empty (smallest file has 46 samples, biggest has around 1200).   
- The next attempt was cutting the files into smaller windows of 46 samples (and potentially padding the last sample of the file). The next cell is the (reduced) output of a run using this strategy.    
- The current one is to cut the files into windows of 10 samples. We'll see the result later.

As the purpose of this classifier is to be a deep neural network, I decided to put two convolutional layers along with two max pooling, followed by two denses layers and an output layer. The whole network can be seen in the following cell.    
Due to the number of parameters for this network (parameters in each layer, layout of the network, etc), most of these are heuristically chosen from a previous project.    
About design decisions :    
- Batch normalization is used for faster and better training
- Prelu activation is used instead of relu for potential better results
- Glorot Normal initialization is used, but it seems that it is a matter of preference between uniform and normal and has no real impact on performance
- Dropouts are used after max pooling and dense layers to reduce overfitting
- Kernel regularizers are used on the convolution and dense layers to reduce overfitting
- (Both dropouts and regularizers were added after checking that the network could learn perfectly the training set)

As expected, the CNN got a really good score for both file and sample accuracy, but it strangely was not perfect like the SNN and is even a bit worse than the random forest. Moreover, the sample accuracy can not really be compared to the one obtained with the previous classifiers, as the CNN has more context (46 times more information than the other classifiers).  
The next run obtains a perfect file score by reducing the window size.

In [4]:
test_classifier(CNNClassifier(verbose=1))

Finished loading/creating features
Using classifier CNNClassifier
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 10, 20, 1)         4         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 18, 32)         320       
_________________________________________________________________
p_re_lu_5 (PReLU)            (None, 8, 18, 32)         4608      
_________________________________________________________________
batch_normalization_5 (Batch (None, 8, 18, 32)         128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 17, 32)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 17, 32)         0         
_________________________________________________________________
conv2d_4 (

39254/39254 [==============================] - 4s 91us/step - loss: 0.4212 - acc: 0.8781 - val_loss: 0.4029 - val_acc: 0.8936
Epoch 38/300
39254/39254 [==============================] - 3s 85us/step - loss: 0.4190 - acc: 0.8767 - val_loss: 0.3841 - val_acc: 0.8847
Epoch 39/300
39254/39254 [==============================] - 4s 90us/step - loss: 0.4189 - acc: 0.8800 - val_loss: 0.3985 - val_acc: 0.8813
Epoch 40/300
39254/39254 [==============================] - 3s 88us/step - loss: 0.4084 - acc: 0.8807 - val_loss: 0.3781 - val_acc: 0.8921
Epoch 41/300
39254/39254 [==============================] - 4s 89us/step - loss: 0.4282 - acc: 0.8789 - val_loss: 0.4001 - val_acc: 0.8857
Epoch 42/300
39254/39254 [==============================] - 3s 86us/step - loss: 0.4293 - acc: 0.8778 - val_loss: 0.3958 - val_acc: 0.8868
Epoch 43/300
39254/39254 [==============================] - 3s 86us/step - loss: 0.4188 - acc: 0.8822 - val_loss: 0.3724 - val_acc: 0.8933
Epoch 44/300
39254/39254 [==============

39254/39254 [==============================] - 3s 86us/step - loss: 0.2347 - acc: 0.9203 - val_loss: 0.2501 - val_acc: 0.9159
Epoch 96/300
39254/39254 [==============================] - 3s 87us/step - loss: 0.2324 - acc: 0.9219 - val_loss: 0.2383 - val_acc: 0.9213
Epoch 97/300
39254/39254 [==============================] - 3s 86us/step - loss: 0.2349 - acc: 0.9221 - val_loss: 0.2432 - val_acc: 0.9174
Epoch 98/300
39254/39254 [==============================] - 4s 90us/step - loss: 0.2321 - acc: 0.9213 - val_loss: 0.2398 - val_acc: 0.9228
Epoch 99/300
39254/39254 [==============================] - 4s 90us/step - loss: 0.2320 - acc: 0.9217 - val_loss: 0.2424 - val_acc: 0.9196
Epoch 100/300
39254/39254 [==============================] - 3s 87us/step - loss: 0.2323 - acc: 0.9228 - val_loss: 0.2389 - val_acc: 0.9219
Epoch 101/300
39254/39254 [==============================] - 3s 87us/step - loss: 0.2317 - acc: 0.9227 - val_loss: 0.2431 - val_acc: 0.9214
Epoch 102/300
39254/39254 [===========

39254/39254 [==============================] - 4s 93us/step - loss: 0.1957 - acc: 0.9363 - val_loss: 0.2166 - val_acc: 0.9306
Epoch 154/300
39254/39254 [==============================] - 3s 88us/step - loss: 0.1959 - acc: 0.9374 - val_loss: 0.2172 - val_acc: 0.9310
Epoch 155/300
39254/39254 [==============================] - 4s 91us/step - loss: 0.1960 - acc: 0.9349 - val_loss: 0.2189 - val_acc: 0.9288
Epoch 156/300
39254/39254 [==============================] - 4s 91us/step - loss: 0.1956 - acc: 0.9368 - val_loss: 0.2230 - val_acc: 0.9269
Epoch 157/300
39254/39254 [==============================] - 4s 91us/step - loss: 0.1942 - acc: 0.9372 - val_loss: 0.2202 - val_acc: 0.9300
Epoch 158/300
39254/39254 [==============================] - 4s 90us/step - loss: 0.1969 - acc: 0.9347 - val_loss: 0.2170 - val_acc: 0.9282
Epoch 159/300
39254/39254 [==============================] - 4s 92us/step - loss: 0.1923 - acc: 0.9376 - val_loss: 0.2167 - val_acc: 0.9310
Epoch 160/300
39254/39254 [=======

Now, by using windows of size 10, we get a the same result as the RandomForest for file accuracy (1 mispredicted file), which is better than a window size of 46. It is probably mainly due to averaging over more samples : As the per sample accuracy is still very high, averaging over them allows for a very good score. In theory, as long as we have a per sample accuracy >50% and a very high number of samples per file, the predictions should be perfect. It is not always the case in practice due to the number of samples per file though. 


100% files accuracy should be obtainable by tuning the parameters or changing the layout.      

Compared to the other classifiers however, it is slower to train, more memory hungry, and therefore seems a bit overkill for this task, given than a simple neural net with a single hidden layer obtains file predictions results better than this one.    

### Dumps size
If we compare the performance/file size metric of the classifiers, we see that the SNN is easily the best one, the 256 units network weighing only 96Ko, compared to the 12Mo for the 10 estimators RandomForest or the 23Mo CNN. The linear SVC only weighs 1Ko, but has bad results.

### Other considerations
- I've also added a cross-validation method to test the classifiers on samples, but due to the time it takes to execute it, I decided to leave it out.
- I have considered data augmentation, but given the results we've already obtained, this isn't at all necessary for this problem and would only be wasted time.

### Conclusion

In conclusion, we can rate the classifiers like this : 
- The winner is obviously the SNN. The complexity of designing it is really low, the file size is also very small, and it gets a perfect result.
- The second one would be the RandomForest. The design complexity is non-existant, the file size is moderately high but still very manageable for a low number of trees, and it gets a nearly perfect result.
- The third is the CNN. Although the design complexity is high due to the number of parameters we can change and layouts we can do, and the file size is higher than the others, it gets a nearly score. It is totally unnecessary to implement such a complex network for this problem though.
- The fourth is the linear SVC, because even though the design complexity and file size is null, the results are bad.
- For obvious reasons, the constant classifier is the last one.

In short, a more complex model is not always better than a simple one.